## Aspect Extraction Using RNN and Tensorflow 

In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import collections
from collections import Counter
import string

### Word and its Vectors 

In [2]:
word_vectors=[]
words=[]
unique_words=[]
vectors = []
f = open('word2vec.csv','r')
word_vectors = f.read()

word_vectors = word_vectors.split('\n',1)[1]
word_vectors = word_vectors.replace(',"',':')
word_vectors = word_vectors.replace('\n','')
word_vectors = word_vectors.split('"')
word_vectors = word_vectors[:-1]
print(len(word_vectors))

129218


In [3]:
word_vectors[0]

'But:[-0.22266774  0.04329333  0.37159213 -0.5329994   0.6239861  -0.24587387  0.34643427  0.13116741 -0.41579244 -0.20065007  0.08610805  0.6125774  0.20295028 -0.12502426 -0.30027688  0.16528115 -0.6238571  -0.03565881 -0.17556378  0.11841667  0.5494477  -0.6912074   0.11358704  0.01227758 -0.3365192  -0.09856282  0.20129     0.29659027 -0.1969936   0.15831183 -0.057485   -0.0241813 ]'

In [4]:
# splitting into individual words and vectors list

for word_vec in word_vectors:
    vectors.append(word_vec.split(':')[1])
    words.append(word_vec.split(':')[0])

In [5]:
print(vectors[0])

[-0.22266774  0.04329333  0.37159213 -0.5329994   0.6239861  -0.24587387  0.34643427  0.13116741 -0.41579244 -0.20065007  0.08610805  0.6125774  0.20295028 -0.12502426 -0.30027688  0.16528115 -0.6238571  -0.03565881 -0.17556378  0.11841667  0.5494477  -0.6912074   0.11358704  0.01227758 -0.3365192  -0.09856282  0.20129     0.29659027 -0.1969936   0.15831183 -0.057485   -0.0241813 ]


In [6]:
#creating a dictionary of unique words

import collections
from collections import Counter

dictionary_words = dict()
for entry in words:
    if entry.lower() not in unique_words:
        unique_words.append(entry.lower())
for word in unique_words:
    dictionary_words[word.lower()] = len(dictionary_words)

#print (len(dictionary_words))  #5257


In [7]:
#dictionary_words

### Labels

In [8]:
f = open('data.csv','r')
temp = f.read()
temp=temp.split('\n')
temp = temp[1:]
print(temp[0])

labels=[]

3121,But the staff was so horrible to us.,staff,negative,8,13,,,,,,,staff,,,,staff 


In [9]:
for tmp in temp:
    tmp = tmp.split(',')
    labels.append(tmp[-1])

In [10]:
labels = labels[:-1]
len(labels)

7408

In [11]:
#creating a list of unique labels

import pandas as pd
df=pd.read_csv('data.csv');
term=df['combine']  # terms are the labels
unique_terms=[]   # new_terms = unique_terms
#term
term = [word.replace(' ','') for word in term]
for entry in term:
    if entry.lower() not in unique_terms:
        unique_terms.append(entry.lower())
        
print(len(term))   #7408
print(len(unique_terms))     #1215 unique labels


7408
1215


In [12]:
#print(unique_terms)

In [13]:
#creating a dictionary of numbers for each label in the corpus

#count = collections.Counter(unique_terms).most_common()
dictionary = dict()
for word in unique_terms:
    dictionary[word.lower()] = len(dictionary)
reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
print (len(reverse_dictionary))  #1215

1215


In [14]:
#dictionary

### x_train

In [15]:
import numpy as np

In [16]:
#creating a integer list of the words from the dictionary

df=pd.read_csv('data.csv');
text=df['text'] 
text_numbers = []
for txt in text:
    #print (text)
    txt=txt.replace('.','')
    txt=txt.replace('comma','')
    txt=txt.replace('\'','')
    all_words=txt.split(' ')
    #print(all_words)
    for i in range(len(all_words)):
        if all_words[i].lower() in dictionary_words:
            all_words[i]=dictionary_words[all_words[i].lower()]
        else:
            all_words[i]=0
    #print(all_words)
    if len(all_words)<32:
        for y in range(32-len(all_words)):
            all_words.append(0)
    elif len(all_words)>32:
        all_words=all_words[0:32]
    text_numbers.append(all_words)

In [17]:
len(text_numbers)

7408

In [18]:
text_numbers = np.array(text_numbers).astype(np.float32)

### y_train

In [19]:
len(term)

7408

In [20]:
y_train = []
for trm in term:
    if trm.lower() in dictionary:
        y_train.append(dictionary[trm.lower()])
    else:
        y_train = -1

In [21]:
len(y_train)

7408

In [22]:
y_train_unique = []
for trm in unique_terms:
    if trm.lower() in dictionary:
        y_train_unique.append(dictionary[trm.lower()])
    else:
        y_train_unique = -1

### RNN

In [23]:
#defining the paramters for the network

num_epochs = 10
total_series_length = 7400
truncated_backprop_length = 32
state_size = 1
num_classes = 1215
echo_step = 3
batch_size = 5
num_batches = int(total_series_length//batch_size//truncated_backprop_length)

#defining the placeholders for the model

batchX_placeholder = tf.placeholder(tf.float32, [batch_size, truncated_backprop_length])
batchY_placeholder = tf.placeholder(tf.int32, [batch_size, truncated_backprop_length])

init_state = tf.placeholder(tf.float32, [batch_size, 1])


# the weight and the bias

W2 = tf.Variable(np.random.rand(state_size, num_classes),dtype=tf.float32)
b2 = tf.Variable(np.zeros((1,num_classes)), dtype=tf.float32)


In [24]:
def train_neural_network(x,y):
    #tf.reset_default_graph()
    
    # Unpack columns
    
    inputs_series = tf.split(batchX_placeholder, truncated_backprop_length, axis=1)
    labels_series = tf.unstack(batchY_placeholder, axis=1)

    # Forward passes
    cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
    states_series, current_state = tf.nn.static_rnn(cell = cell, inputs = inputs_series,initial_state = init_state)
        
    logits_series = [tf.matmul(state, W2) + b2 for state in states_series] #Broadcasted addition
    predictions_series = [tf.nn.softmax(logits) for logits in logits_series]
    

    losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels) for logits, labels in zip(logits_series,labels_series)]
    total_loss = tf.reduce_mean(losses)

    train_step = tf.train.AdagradOptimizer(0.3).minimize(total_loss)
   
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        loss_list = []

        for epoch_idx in range(num_epochs):
            _current_state = np.zeros((batch_size, state_size))

            print("\nNew data, epoch", epoch_idx)

            for batch_idx in range(num_batches):
                start_idx = batch_idx * truncated_backprop_length
                end_idx = start_idx + truncated_backprop_length
                
                batchX = x[:,start_idx:end_idx]
                batchY = y[:,start_idx:end_idx]

                _total_loss, _train_step, _current_state, _predictions_series = sess.run(
                    [total_loss, train_step, current_state, predictions_series],
                    feed_dict={
                        batchX_placeholder:batchX,
                        batchY_placeholder:batchY,
                        init_state:_current_state
                    })

                loss_list.append(_total_loss)

                if batch_idx%20 == 0:
                    print("Step",batch_idx, "Loss", _total_loss)
        

In [25]:
text_numbers=text_numbers[0:7400]
y_train = y_train[0:7400]
tn_copy = text_numbers
tn_copy = tn_copy.reshape([batch_size,-1])
lb_copy = []
count = int(len(y_train)/batch_size)  #1480
cnt = 0
j = 0

while j < batch_size:
    lb = []
    for counter in range(count):
        lb.append(y_train[cnt+counter])
        
    lb_copy.append(lb)
    cnt+=count
    j+=1
lb_copy = np.array(lb_copy)

train_neural_network(tn_copy,lb_copy)

print("\nTraining done");

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Use `tf.global_variables_initializer` instead.

New data, epoch 0
Step 0 Loss 7.1242743
Step 20 Loss 5.190398
Step 40 Loss 4.7921286

New data, epoch 1
Step 0 Loss 4.620501
Step 20 Loss 3.9734256
Step 40 Loss 4.4526715

New data, epoch 2
Step 0 Loss 4.398009
Step 20 Loss 3.8414574
Step 40 Loss 4.3799043

New data, epoch 3
Step 0 Loss 4.3495817
Step 20 Loss 3.7875156
Step 40 Loss 4.330928

New data, epoch 4
Step 0 Loss 4.3152785
Step 20 Loss 3.7496612
Step 40 Loss 4.2923374

New data, epoch 5
Step 0 Loss 4.2882667
Step 20 Loss 3.72107
Step 40 Loss 4.2620177

New data, epoch 6
Step 0 Loss 4.2671094
Step 20 Loss 3.6993968
Step 40 Loss 4.238047

New data, epoch 7
Step 0 Loss 4.250466
Step 20 Loss 3.6828933
Step 40 Loss 4.21874

New data, epoch 8
Step 0 Loss 4.237222
Step 20 Loss 3.670138
Step 40 Loss 4.2028575

New data, epoch 9
St